In [21]:
import pandas as pd
import json
import random

In [9]:
# Load entity_params into a dictionary

print("Started Reading JSON file")
with open("entity_params.json", "r") as read_file:
    print("Converting JSON encoded data into Python dictionary")
    entity_params_dict = json.load(read_file)

Started Reading JSON file
Converting JSON encoded data into Python dictionary


In [37]:
# set paramters
num_biz = 1000
criminal_biz_proportion = 0.01 # proportion of criminal businesses

In [40]:
# Create num_biz business ids
entity_id = ["B_" + str(x) for x in range(1, num_biz + 1)]

In [49]:
# Create business region by sampling from region distribution in param dict
total_biz = sum(entity_params_dict["business"]["region_dist"].values())
region_list = list(entity_params_dict["business"]["region_dist"].keys())
region_weights = list(entity_params_dict["business"]["region_dist"].values())

region = random.choices(region_list, weights=region_weights, k=num_biz)

In [50]:
# Create business sector by sampling from conditional distribution of sector
# given region from param dict

sector = []
for rgn in region:
    sector_list = list(entity_params_dict["business"]["region_sector_dist"][rgn].keys())
    sector_weights = list(entity_params_dict["business"]["region_sector_dist"][rgn].values())
    sector.append(random.choices(sector_list, weights = sector_weights, k=1)[0])
    

In [73]:
# Create business sector by sampling from conditional distribution of turnover
# given region from param dict

turnover = []
for rgn in region:
    turnover_list = list(entity_params_dict["business"]["region_turnover_dist"][rgn].keys())
    turnover_weights = list(entity_params_dict["business"]["region_turnover_dist"][rgn].values())
    turnover.append(random.choices(turnover_list, weights = turnover_weights, k=1)[0])

turnover = [x.replace('+','-1000000') for x in turnover]
turnover = [x.split('-') for x in turnover]
turnover = [int(random.uniform(int(x[0]),int(x[1]))) * 1000 for x in turnover]

In [52]:
# Create criminal flag
criminal = random.choices([0,1], weights = [1-criminal_biz_proportion, criminal_biz_proportion], k=num_biz)

In [53]:
# Create account ids, for now each business has a current and a loan account
# account ids are just the entity_id with the suffic CUR or LOA

account_ids = []
for entity in entity_id:
    account_ids.append([str(entity) + '_CUR', str(entity) + '_LOA'])

In [54]:
business_df = pd.DataFrame({
    "entity_id" : entity_id,
    "region" : region,
    "sector" : sector,
    "turnover" : turnover,
    "account_ids" : account_ids,
    "criminal" : criminal
})

In [55]:
business_df.head()

,entity_id,region,sector,turnover,account_ids,criminal
0,B_1,London,Arts entertainment recreation other services,100-249,"[B_1_CUR, B_1_LOA]",0
1,B_2,East,Professional scientific technical,100-249,"[B_2_CUR, B_2_LOA]",0
2,B_3,North West,Health,250-499,"[B_3_CUR, B_3_LOA]",0
3,B_4,Wales,Construction,50-99,"[B_4_CUR, B_4_LOA]",0
4,B_5,Scotland,Accommodation food services,100-249,"[B_5_CUR, B_5_LOA]",0
